In [207]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
from datetime import datetime, timedelta
import time

import os
import sys

from google.cloud import bigquery
from google.oauth2 import service_account
import gspread
from gspread_dataframe import set_with_dataframe
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload

import warnings

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"
sheetskey_path = KEY_PATH + "mido-project-426906-41a4b6d0e3db.json"
midopluskey_path = KEY_PATH + "midoplus.json"

warnings.filterwarnings("ignore")

In [3]:
def get_service_key(servicekey_path):

    with open(servicekey_path) as f:
        data = json.load(f)
    return data

In [5]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [6]:
create_bigquery_client(midopluskey_path)

In [7]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

#### 구글 스프레드 시트

In [16]:
# Google Sheets 클라이언트 생성
sheets_scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

sheets_creds = service_account.Credentials.from_service_account_file(midopluskey_path, scopes=sheets_scope)
gc = gspread.authorize(sheets_creds)

#### 생산일지 
- 원본 생산일지 로드해서 백업용 생성
- 원본생산일지 추가해서 --> production_report_백업

In [11]:
doc_path = 'C:/py_src/midoproject/data/production_report/'

all_doc_data = []
for doc in os.listdir(doc_path):
    if 'all_doc_df_final' not in doc:
        doc_df = pd.read_excel(doc_path + doc, header=5)

        # 불필요데이터 제거
        doc_df = doc_df.iloc[:doc_df['R/NO'].last_valid_index()+1]

        # 모든 컬럼 NaN 제거
        doc_df = doc_df.dropna(how='all')

        # 컬럼형식 맞추기
        doc_df = doc_df.rename(columns={'날짜':'생산일자', '일자':'생산일자'})
        doc_df = doc_df.rename(columns={'제직량 M':'제직량', '입고량 M':'입고량'})

        # 일자, 구장명 채우기
        # doc_df['계약코드'] = doc_df['계약코드'].ffill()
        doc_df['생산일자'] = doc_df['생산일자'].ffill()
        doc_df['구장명'] = doc_df['구장명'].ffill()
        
        # 데이터프레임을 리스트에 추가
        all_doc_data.append(doc_df)

In [12]:
# 리스트의 모든 데이터프레임을 하나로 병합
all_doc_df_fin = pd.concat(all_doc_data, ignore_index=True)

In [13]:
# '일자'부터 'YARN'까지의 컬럼을 '구장명' 기준 첫 행으로 채우기
columns_to_fill = ['계약코드','오더량', 'ITEM', 'SPI', 'PH', 'PW', 'T/F TYPE', '기포지', '제직폭', '본수', '입고폭', 'YARN', 'COLOR', '도전사','R/NO']
for col in columns_to_fill:
    all_doc_df_fin[col] = all_doc_df_fin.groupby(['생산일자','구장명'])[col].transform(lambda x: x.fillna(x.iloc[0]))

In [14]:
# 제직량, 입고량 없는거 제거
all_doc_df_final = all_doc_df_fin[~all_doc_df_fin['제직량'].isnull()].drop_duplicates().reset_index(drop=True)

In [ ]:
# all_doc_df_final.to_csv('C:/py_src/midoproject/data/2024/all_doc_df_final.csv',encoding='utf-8-sig',index=False)
## 원본 생산일지 불러와서 전처리 후 통합
## 생산일지 백업용 데이터 생성
## 스프레드시트 'production_report_백업' 시트에 직접 추가

구글스프레드시트 : production_report_백업 -> production_report

In [141]:
# 스프레드시트 ID (URL에서 확인 가능)
production_sheet_id = '1dl6GyEkzarfgcKSmT9hwYzR63MhcGDox0z-gCa0_7OI' ## midoplus production_report --> 생산일지통합
spreadsheet_production = gc.open_by_key(production_sheet_id)

# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'production_report_백업'
worksheet_production = spreadsheet_production.worksheet(WORKSHEET_NAME)

# 구글 시트에서 데이터 읽기
data = worksheet_production.get_all_records()
production_df_backup = pd.DataFrame(data)

In [142]:
def convert_date_format(date_str):
    if isinstance(date_str, str):
        if len(date_str.split('.')) == 3:
            # 기존 형식이 '연도. 월. 일' 인 경우
            return pd.to_datetime(date_str, format='%Y.%m.%d').strftime('%Y-%m-%d')
        elif len(date_str.split('.')) == 2:
            # 새 형식이 '월.일' 인 경우, 연도를 '2024'로 가정
            return pd.to_datetime('2024.' + date_str, format='%Y.%m.%d').strftime('%Y-%m-%d')
    return date_str  # 변환할 수 없는 경우 또는 비문자형 데이터인 경우 원본 반환

In [143]:
# '생산일자' 컬럼에 변환 함수 적용
production_df_backup = production_df_backup.astype({'생산일자':str,'납품요구접수일자':str,'납품기한일자':str})
production_df_backup['생산일자'] = production_df_backup['생산일자'].str.split('~').str[0] 
production_df_backup['생산일자'] = production_df_backup['생산일자'].str.replace(' ','').str.replace('0:00','').str.replace('2024-05-078','2024-05-08') 
production_df_backup['생산일자'] = production_df_backup['생산일자'].apply(convert_date_format)
production_df_backup['납품요구접수일자'] = production_df_backup['납품요구접수일자'].str.replace(' ','').str.replace('0:00','')
production_df_backup['납품요구접수일자'] = production_df_backup['납품요구접수일자'].apply(convert_date_format)
production_df_backup['납품기한일자'] = production_df_backup['납품기한일자'].str.replace(' ','').str.replace('0:00','')
production_df_backup['납품기한일자'] = production_df_backup['납품기한일자'].apply(convert_date_format)

# 구글시트 업데이트 위해 NaN값 대체
production_df_backup = production_df_backup.fillna('')

# 계약코드 대체
production_df_backup['계약코드'] = np.where(production_df_backup['계약코드']=='',production_df_backup['납품요구번호'],production_df_backup['계약코드']) ## 계약코드 없는것 --> 납품요구번호로 대체
production_df_backup['계약코드'] = production_df_backup['계약코드'].astype(str)
production_df_backup['계약코드'] = production_df_backup['계약코드'].str.replace('-','')

# 제품 오입력 수정
production_df_backup['ITEM'] = production_df_backup['ITEM'].str.replace(',','')

# 납품요구건명 수정
production_df_backup['납품요구건명'] = production_df_backup['납품요구건명'].apply(lambda x: ' '.join(x.split('\n')[1].strip().split(' ')[1:]) if '\n' in x else x) ## 납품요구건명 오정보 수정

In [144]:
# 계약코드 임의 설정
production_df_backup['계약코드'] = np.where(production_df_backup['계약코드']=='',production_df_backup.index,production_df_backup['계약코드'])
production_df_backup['계약코드'] = production_df_backup.groupby('구장명')['계약코드'].transform('first')

In [145]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'production_report'
worksheet_production = spreadsheet_production.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_production.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_production.update([production_df_backup.columns.values.tolist()] + production_df_backup.values.tolist())

{'spreadsheetId': '1dl6GyEkzarfgcKSmT9hwYzR63MhcGDox0z-gCa0_7OI',
 'updatedRange': 'production_report!A1:X9040',
 'updatedRows': 9040,
 'updatedColumns': 24,
 'updatedCells': 216960}

#### 계약현황

In [193]:
# 생산일지 -> 계약현황
contract_df = production_df_backup.copy()

In [194]:
# 전처리
contract_df = contract_df.drop_duplicates(['납품요구번호','계약코드','납품요구건명','구장명']).reset_index(drop=True) ## 중복제거

In [195]:
# 날짜 관련 전처리
contract_df['납품기한일자'] = pd.to_datetime(contract_df['납품기한일자'])
contract_df['생산일자'] = pd.to_datetime(contract_df['생산일자'])

contract_df['납품기한일자'] = np.where(contract_df['납품기한일자'].isnull(), contract_df['생산일자'] + DateOffset(months=3), contract_df['납품기한일자'])
## 계약코드별 생산일자부터 납품기한일자까지 평균
## 생산일자부터 납품기한일자까지 평균 소요일 : 89.31813637293709일

# 현재 날짜 기준으로 진행상황 컬럼 추가
current_date = datetime.now()
contract_df['납품기한일자'] = pd.to_datetime(contract_df['납품기한일자'])
contract_df['진행상황'] = contract_df['납품기한일자'].apply(lambda x: '완료' if x < current_date else '진행중')
contract_df['납품기한일자'] = contract_df['납품기한일자'].astype(str)

In [196]:
# 계약현황 데이터셋 생성
contract_df_fin = contract_df[['납품요구번호', '계약코드', '납품요구접수일자', '납품기한일자', '납품요구건명', '구장명', '진행상황']] # '견적서', '발주서', '도면'
contract_df_fin['견적서'] = None
contract_df_fin['발주서'] = None
contract_df_fin['도면'] = None

In [199]:
# 스프레드시트 ID (URL에서 확인 가능)
contracts_sheet_id = '1YEl1Sw_uKw8URp8_nOfmgxBQWJtuiVJcH9ZYMHszGH0' ## midoplus contracts
spreadsheet_contracts = gc.open_by_key(contracts_sheet_id)

In [200]:
# 기존 스프레드시트 및 워크시트 열기
WORKSHEET_NAME = 'contracts'
worksheet_contracts = spreadsheet_contracts.worksheet(WORKSHEET_NAME)

# 기존 데이터 지우기
worksheet_contracts.clear()

# 데이터 프레임을 스프레드 시트에 덮어쓰기
worksheet_contracts.update([contract_df_fin.columns.values.tolist()] + contract_df_fin.values.tolist())

{'spreadsheetId': '1YEl1Sw_uKw8URp8_nOfmgxBQWJtuiVJcH9ZYMHszGH0',
 'updatedRange': 'contracts!A1:J884',
 'updatedRows': 884,
 'updatedColumns': 10,
 'updatedCells': 6191}